# Initial setup

In [4]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections
import pickle

import tweepy as tw
import nltk
from nltk.corpus import stopwords
import re
import networkx

import warnings
warnings.filterwarnings("ignore")

In [5]:
# Load saved keys from API key file
keys = pickle.load(open('api_key.p', 'rb'))
auth = tw.OAuthHandler(keys['consumer_key'], keys['consumer_secret'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tw.API(auth, wait_on_rate_limit=True)

# Get trending topics

In [14]:
# See available regions
# api.trends_available() # worldwide has woeid (Where On Earth IDentifier) of 1; US of 23424977

# Get worldwide trending topics
trends1 = api.trends_place(23424977) # 23424977 is the US

# Extract topic name and number of tweets for each
trends = [(trend['name'], trend['tweet_volume']) for trend in trends1[0]['trends']]

# Create pandas df
trends_df = pd.DataFrame(trends, columns=['Topic', 'Count'])
trends_df.head(10)

,Topic,Count
0,Jersey City,69183.0
1,#BROCKHAMPTON,24373.0
2,Mac Pro,15685.0
3,Barr,552681.0
4,#ripnickcannon,63116.0
5,#ImpeachAndConvictTrump,33755.0
6,#HumanRightsDay,201591.0
7,Lauren London,18961.0
8,#TuesdayThoughts,74824.0
9,USMCA,171813.0


# Use API to download tweets

In [7]:
# Download tweets based on a specific trending topic using API (slow)

# Specify topic of interest
topic = trends_df.Topic[0] # Jersey City
search_term = topic + ' -filter:retweets'

# Get tweets
tweets = tw.Cursor(api.search,
                   q=search_term,
                   lang="en",
                   since='2019-12-09').items(10) # Only 10 for now

all_tweets = [tweet.text for tweet in tweets]

all_tweets[:5]

["⌚️ HALF TIME'\n\n⚡️ LYON 🆚 RB LEIPZIG\n#⃣ #OLRBL   0-2\n\n⚡️ BENFICA 🆚 ZÉNITH \n#⃣ #SLBZSP 0-0\n\n⚡️ BORUSSIA DORTMUND 🆚 S… https://t.co/zGA6cV5bdX",
 '⏰ HALF-TIME | #UCL ⏰\n\n#BVBSLA 1-1\n#InterBarça 1-1\n#OLRBL 0-2\n#SLBZenit 0-0\n#AJAVAL 0-1\n#CHELOSC 2-0\n\nThe best life… https://t.co/ECOTEpm1xq',
 'Big second half needed.\n\n#UCL #OLRBL https://t.co/TZCyglKKqh',
 "⌚️ LIVE 45+4'\n\n⚡️ LYON 🆚 RB LEIPZIG\n#⃣ #OLRBL   0-2\n\n⚡️ BENFICA 🆚 ZÉNITH \n#⃣ #SLBZSP 0-0\n\n⚡️ BORUSSIA DORTMUND 🆚 S… https://t.co/tWSEMrwkWZ",
 'HT: Lyon 0-2 Leipzig\n\nNot a massive amount between the sides, but 2 silly penalties have the visitors on easy stree… https://t.co/0tRTFVHU4u']

# GetOldTweets to download tweets
Based on: https://github.com/fajarmf10/GetOldTweets-python and https://github.com/Jefferson-Henrique/GetOldTweets-python/issues/52#issuecomment-276932885

Steps taken:
- git clone https://github.com/Jefferson-Henrique/GetOldTweets-python
- cd GetOldTweets-python
- mv got3 ~/anaconda3/lib/python3.7/site-packages
- pip install pyquery

In [3]:
# Get old tweets
import got3

In [31]:
# Download tweets using GetOldTweets instead of API
# Specify trending topic of interest
topic = trends_df.Topic[0] # Jersey City
numTweets = 50 # How many tweets to scrape
tweetCriteria = got3.manager.TweetCriteria().setQuerySearch(topic).setLang('english').setSince('2019-12-10').setUntil('2019-12-11').setMaxTweets(numTweets)

# Get tweets
tweets = got3.manager.TweetManager.getTweets(tweetCriteria)

# Create pandas df
all_tweets_df = pd.DataFrame([], columns=['ID', 'Link', 'Username', 'Text',
                                          'Date', 'Retweets', 'Favorites',
                                          'Mentions', 'Hashtags', 'Geo',
                                          'URLs', 'Author_ID'])
for i in range(numTweets):
    all_tweets_df.loc[i] = [tweets[i].id, tweets[i].permalink, tweets[i].username, tweets[i].text,
                            tweets[i].date, tweets[i].retweets, tweets[i].favorites,
                            tweets[i].mentions, tweets[i].hashtags, tweets[i].geo,
                            tweets[i].urls, tweets[i].author_id]
all_tweets_df.head(20)

,ID,Link,Username,Text,Date,Retweets,Favorites,Mentions,Hashtags,Geo,URLs,Author_ID
0,1204507349370724352,https://twitter.com/fetchfeeds/status/12045073...,,Jersey City . Check out our articles here! htt...,2019-12-10 16:05:05,0,0,,#Jersey,,https://fetchfeeds.com/discover?q=Jersey%20City,876540937496805376
1,1204507341418323974,https://twitter.com/Spencz1997/status/12045073...,,"Stay safe, Jersey City .",2019-12-10 16:05:03,0,0,,,,,272152883
2,1204507340831113219,https://twitter.com/cerritosnews/status/120450...,,Black-clad couple open fire on Jersey City sid...,2019-12-10 16:05:03,0,0,,,,http://www.loscerritosnews.net/2019/12/10/blac...,34072567
3,1204507334334189568,https://twitter.com/Trending_NewX/status/12045...,,BNONews : BREAKING: Officials confirm 1 office...,2019-12-10 16:05:01,0,0,,,,https://buff.ly/2rxMZpf,550933192
4,1204507332710977538,https://twitter.com/PaulaSlier_RT/status/12045...,,#JerseyCity : What a bizarre story this Jersey...,2019-12-10 16:05:01,0,0,,#JerseyCity #shootout #police #kosher,,,388900759
5,1204507331775680513,https://twitter.com/shopbuypay/status/12045073...,,He was involved in the initial shooting #scann...,2019-12-10 16:05:01,0,0,,#scanner #JerseyCity,,,1068927002730733568
6,1204507323298983936,https://twitter.com/ChantalAlexa_/status/12045...,,"Praying for everyone in jersey city Rn, smh gu...",2019-12-10 16:04:59,0,0,,,,,801453440
7,1204507318328709122,https://twitter.com/Satmar550/status/120450731...,,"6 confirmed dead in Jersey City mass shooting,...",2019-12-10 16:04:58,0,0,,,,,780807778080215040
8,1204507312825753602,https://twitter.com/nataliaOJ321/status/120450...,,pipe bomb confirmed by #jerseycity Police in u...,2019-12-10 16:04:56,0,0,,#jerseycity,,,33566272
9,1204507309239685120,https://twitter.com/lehighvalley/status/120450...,,"Police officer dies in Jersey City shooting, 2...",2019-12-10 16:04:55,0,0,,,,https://trib.al/0Pa6fkI,17063785


# Basic EDA of 'Text'

In [32]:
# Function to standardize text (make lowercase and remove URLs, mentions, and non-alphanumeric characters)
def standardize_text(df):
    df = df.str.lower() # Make everything lowercase
    df = df.str.replace(r"http\S+", "") # Remove URLs
    df = df.str.replace(r"http", "") # Remove "http"s
    df = df.str.replace(r"pic.twitter.com\S+", "") # Remove "pic.twitter.com"s
    df = df.str.replace(r"@\S+", "") # Remove mentions
    df = df.str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ") # Remove any non-alphanumeric chars
    df = df.str.replace(r"@", "at") # Replace "@" with "at"
    return df

# Clean up 'Text'
clean_tweets = standardize_text(all_tweets_df['Text'])
clean_tweets[:5]

0    jersey city   check out our articles here!  fe...
1                             stay safe, jersey city  
2    black clad couple open fire on jersey city sid...
3    bnonews   breaking  officials confirm 1 office...
4     jerseycity   what a bizarre story this jersey...
Name: Text, dtype: object

In [33]:
# Create a list of lists containing words for each tweet
words_in_tweet = [tweet.split() for tweet in clean_tweets]

# Set stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

# Add query words to stopwords
stop_words.extend(topic.lower().split(' '))

# Remove stopwords
tweets_nsw = [[word for word in tweet_words if word not in stop_words]
              for tweet_words in words_in_tweet]

# Determine all words used
all_words_nsw = list(itertools.chain(*tweets_nsw))

# Create counter of words in tweets
counts_nsw = collections.Counter(all_words_nsw)

# Create dataframe for counter
clean_tweets_df = pd.DataFrame(counts_nsw.most_common(15),
                             columns=['words', 'count'])
clean_tweets_df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/amywen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,words,count
0,police,19
1,shooting,16
2,officer,14
3,com,13
4,jerseycity,13


In [34]:
# Plot horizontal bar graph of most frequent words
fig, ax = plt.subplots(figsize=(8, 8))

clean_tweets_df.sort_values(by='count').plot.barh(x='words',
                      y='count',
                      ax=ax,
                      color="purple")

ax.set_title("Most Common Words Found in Tweets")

plt.show()